This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
from collections import OrderedDict
from statistics import mean, median

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
data_sample = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?limit=1&api_key=" + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data_sample_json = data_sample.json()
data_sample_json

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2018-09-10T22:28:18.905Z',
  'newest_available_date': '2018-09-10',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': 1,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2018-09-10',
  'data': [['2018-09-10',
    None,
    76.25,
    75.3,
    76.15,
    None,
    74999.0,
    5691773.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 

In [5]:
afx_data_2017 = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY)
afx_json = afx_data_2017.json()

## Convert the returned JSON object into a Python dictionary.

In [6]:
# Taking the columns name from json object
cols = afx_json['dataset']['column_names']
cols = cols[1:]
cols

['Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [7]:
# Taking the numbers from the dataset of json object
numbers = afx_json['dataset']['data']
# Preview of numbers
numbers[0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [8]:
afx_dict = {day[0]:dict(zip(cols, day[1:])) for day in numbers}
afx_dict = OrderedDict(afx_dict.items())

In [9]:
# Preview of the resulting dictionary structure
afx_dict['2017-12-28']

{'Open': 51.65,
 'High': 51.82,
 'Low': 51.43,
 'Close': 51.6,
 'Change': None,
 'Traded Volume': 40660.0,
 'Turnover': 2099024.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

## Calculate what the highest and lowest opening prices were for the stock in this period.

In [10]:
min_open_price = min(day["Open"] for day in afx_dict.values() if day["Open"] != None)
min_open_price

34.0

In [11]:
max_open_price = max(day["Open"] for day in afx_dict.values() if day["Open"] != None)
max_open_price

53.11

## What was the largest change in any one day (based on High and Low price)

In [12]:
largest_daily_change = max(day["High"] - day["Low"] for day in afx_dict.values() if (day["High"] != None))
largest_daily_change

2.8100000000000023

## What was the largest change between any two days (based on Closing Price)?

In [13]:
def largest_change_two_days(d):
    largest_change = 0
    next_day_price = d.popitem(last=False)[1]["Close"]
    for data in d.values():
        closing_price = data["Close"]
        largest_change = max(largest_change, abs(closing_price - next_day_price))
        next_day_price = closing_price
    return largest_change

In [14]:
largest_change_2days = largest_change_two_days(afx_dict)
largest_change_2days

2.559999999999995

## What was the average daily trading volume during this year?

In [15]:
average_daily_volume = mean(day["Traded Volume"] for day in afx_dict.values())
average_daily_volume

89338.84251968504

## What was the median trading volume during this year?

In [16]:
median_volume = median(day["Traded Volume"] for day in afx_dict.values())
median_volume

76443.0